In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import AFQ.registration as reg
import AFQ.utils.models as ut
import AFQ.data as afd
import AFQ.tractography as aft
import AFQ.csd as csd
import AFQ.dti as dti
import dipy.data as dpd
import nibabel as nib

In [3]:
img, gtab = dpd.read_stanford_hardi()

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/.dipy/stanford_hardi 


In [4]:
afq_templates = afd.read_templates()

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/AFQ_data/templates 


In [5]:
afq_templates

{'ATR_roi1_L': <nibabel.nifti1.Nifti1Image at 0x11b4502b0>,
 'ATR_roi1_R': <nibabel.nifti1.Nifti1Image at 0x11b44c2b0>,
 'ATR_roi2_L': <nibabel.nifti1.Nifti1Image at 0x11b456400>,
 'ATR_roi2_R': <nibabel.nifti1.Nifti1Image at 0x11b44cfd0>,
 'CGC_roi1_L': <nibabel.nifti1.Nifti1Image at 0x11b45a550>,
 'CGC_roi1_R': <nibabel.nifti1.Nifti1Image at 0x11b450da0>,
 'CGC_roi2_L': <nibabel.nifti1.Nifti1Image at 0x11b456d30>,
 'CGC_roi2_R': <nibabel.nifti1.Nifti1Image at 0x11b450c18>,
 'CST_roi1_L': <nibabel.nifti1.Nifti1Image at 0x11b44c048>,
 'CST_roi1_R': <nibabel.nifti1.Nifti1Image at 0x11b450438>,
 'CST_roi2_L': <nibabel.nifti1.Nifti1Image at 0x11b4508d0>,
 'CST_roi2_R': <nibabel.nifti1.Nifti1Image at 0x11b456048>,
 'FA_L': <nibabel.nifti1.Nifti1Image at 0x11b44ca58>,
 'FA_R': <nibabel.nifti1.Nifti1Image at 0x11b45a390>,
 'FP_L': <nibabel.nifti1.Nifti1Image at 0x11b450a58>,
 'FP_R': <nibabel.nifti1.Nifti1Image at 0x11b44cc18>,
 'HCC_roi1_L': <nibabel.nifti1.Nifti1Image at 0x11b44c5c0>,
 'HC

In [ ]:
MNI_T2 = dpd.read_mni_template()
MNI_T2_data = MNI_T2.get_data()
MNI_T2_affine = MNI_T2.get_affine()
dwi = img
dwi_affine = dwi.get_affine()
dwi_data = dwi.get_data()
mean_b0 = np.mean(dwi_data[..., gtab.b0s_mask], -1)
warped_b0, mapping = reg.syn_registration(mean_b0, MNI_T2_data,
                                          moving_affine=dwi_affine,
                                          static_affine=MNI_T2_affine,
                                          step_length=0.1,
                                          sigma_diff=2.0,
                                          metric='CC',
                                          dim=3,
                                          level_iters=[10, 10, 5],
                                          prealign=None)



Data size is approximately 35MB
Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/.dipy/mni_template 
Creating scale space from the moving image. Levels: 3. Sigma factor: 0.200000.
Creating scale space from the static image. Levels: 3. Sigma factor: 0.200000.
Optimizing level 2
Optimizing level 1
Optimizing level 0

In [ ]:
warped_b0.shape

In [ ]:
mean_b0.shape

In [ ]:
CST_L1 = afq_templates['CST_roi1_L']
CST_L2 = afq_templates['CST_roi2_L']

In [ ]:
CST_L1_data = CST_L1.get_data()
CST_L2_data = CST_L2.get_data()

In [ ]:
warped_CST_L1 = mapping.transform_inverse(CST_L1_data, interpolation='nearest')

In [ ]:
from dipy.segment.mask import median_otsu

In [ ]:
masked_b0, brain_mask = median_otsu(mean_b0)

In [ ]:
plt.matshow(brain_mask[:, :, 40])

In [ ]:
nib.save(nib.Nifti1Image(brain_mask.astype(int), img.affine), 'brain_mask')

In [ ]:
#dti.fit_dti(['/Users/arokem/.dipy/stanford_hardi/HARDI150.nii.gz'], 
#            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bval'], 
#            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bvec'], 
#            out_dir='.', mask='brain_mask.nii')

In [ ]:
#nib.save(nib.Nifti1Image((nib.load('dti_FA.nii.gz').get_data() > 0.2).astype(int), img.affine), 'wm_mask.nii.gz')

In [ ]:
#csd.fit_csd(['/Users/arokem/.dipy/stanford_hardi/HARDI150.nii.gz'], 
#            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bval'], 
#            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bvec'], 
#            out_dir='.',
#            mask='brain_mask.nii')

In [ ]:
mask = nib.load('wm_mask.nii.gz').get_data()

In [ ]:
plt.matshow(mask[:, :, 40])

In [ ]:
csd_coeff = nib.load('csd_sh_coeff.nii.gz').get_data()

In [ ]:
plt.matshow(csd_coeff[:,:,40, 0])

In [ ]:
#localtracking = aft.track('./csd_sh_coeff.nii.gz', seed_density=[1, 1, 1],
#                          seed_mask=nib.load('wm_mask.nii.gz').get_data(), 
#                          stop_mask=nib.load('wm_mask.nii.gz').get_data())

In [60]:
#streamlines = list(localtracking)

In [65]:
#sl_long = [sl for sl in streamlines if sl.shape[0]>3]

In [ ]:
from AFQ.utils.streamlines import write_trk

In [ ]:
write_trk('./csd_streamlines.trk', sl_long)

array([[ -6.05825098e+01,  -5.73190608e+01,   2.04144600e+01],
       [ -6.01661569e+01,  -5.72341160e+01,   2.06779646e+01],
       [ -5.97813161e+01,  -5.71022099e+01,   2.09686506e+01],
       [ -5.94332516e+01,  -5.69233425e+01,   2.12798700e+01],
       [ -5.90983413e+01,  -5.66685083e+01,   2.15498600e+01],
       [ -5.87634310e+01,  -5.64136740e+01,   2.18198499e+01],
       [ -5.84747913e+01,  -5.61118785e+01,   2.20948164e+01],
       [ -5.81712656e+01,  -5.57810773e+01,   2.23149137e+01],
       [ -5.78677400e+01,  -5.54502762e+01,   2.25350110e+01],
       [ -5.75642144e+01,  -5.51194751e+01,   2.27551083e+01],
       [ -5.72606888e+01,  -5.47886740e+01,   2.29752056e+01],
       [ -5.69571632e+01,  -5.44578729e+01,   2.31953030e+01],
       [ -5.66536375e+01,  -5.41270718e+01,   2.34154003e+01],
       [ -5.63501119e+01,  -5.37962707e+01,   2.36354976e+01],
       [ -5.59492869e+01,  -5.35303867e+01,   2.37720442e+01],
       [ -5.55132593e+01,  -5.33114641e+01,   2.3881373

In [ ]:
## For the corticospinal tract, need to potentially deal with fibers that loop through the brainstem

## Deal with fiber outliers

## Compare with the track probability maps? 